In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data and Preprocessing

In [ ]:
# Keep only needed columns
feature_names = [
    "Age", "Workclass", "Education", 
    "Occupation", "Relationship", "Race", 
    "Sex", "Capital Gain", "Capital Loss", 
    "Hours per week", "Country", "label"
]

df = pd.DataFrame(
  np.genfromtxt('dataset/adult.csv', delimiter=', ', dtype=str, usecols=(0,1,3,6,7,8,9,10,11,12,13,14)),    
  columns=feature_names
)

# Drop missing values denoted as ?
cols = list(df.columns)
df[cols] = df[cols].replace(['?'], np.nan)
df = df.dropna()


# Declare categorical columns
categorical_columns=[
   "Workclass", "Education", "Occupation", "Relationship", "Race", "Sex", "Country", "label"
]

# Assign type to features
for feature in feature_names:
    if feature in categorical_columns:
        df[feature] = df[feature].astype("category")
    else:
        df[feature] = df[feature].astype("int")

# Make bins for age and hours per week
df['Age'] = pd.cut(df['Age'], bins=[16, 35 , 90],labels=['Young','Aged'])
df["Hours per week"] = pd.cut(
            x=df["Hours per week"],
            bins=[0.9, 25, 39, 40, 55, 100],
            labels=["PartTime", "MidTime", "FullTime", "OverTime", "BrainDrain"])

# Replace "Husband" and "Wife" values with the unisex "Married"
df["Relationship"] = df["Relationship"].replace('Husband','Married')
df["Relationship"] = df["Relationship"].replace('Wife','Married')

# Will have income >50k (1 or 0)
df["label"] = df["label"].replace('>50K',1)
df["label"] = df["label"].replace('<=50K',0)

df.sample(5)

In [ ]:
df.info()

## Model Creation

In [ ]:
# Data preprocessing to train model
from omnixai.data.tabular import Tabular
from omnixai.preprocessing.tabular import TabularTransform
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

tabular_data = Tabular(
   df,
   categorical_columns=[
  "Age", "Workclass", "Education", "Occupation", "Relationship", "Race", "Sex", "Hours per week", "Country"
],
   target_column='label'
)
transformer = TabularTransform().fit(tabular_data)
class_names = transformer.class_names
x = transformer.transform(tabular_data)

# Split data into training and (validation + test) datasets
train, X_temp, train_labels, y_temp  = \
    train_test_split(x[:, :-1], x[:, -1], train_size=0.70, random_state = 123)

# Split data validation and test sets
val, test, val_labels, test_labels = train_test_split(X_temp, y_temp, test_size=0.5, random_state=123)

test_labels = test_labels.astype(int)

print('Training data shape:   {}'.format(train.shape))
print('Validation data shape:  {}'.format(val.shape))
print('Test data shape:        {}'.format(test.shape))

# Train a Random Forest model
model = RandomForestClassifier(random_state=123)
model.fit(train, train_labels)

predict_function=lambda z: model.predict_proba(transformer.transform(z))

# Convert the transformed data back to Tabular instances
train_data = transformer.invert(train)
test_data = transformer.invert(test)

display(tabular_data.target_column)
display(train_labels[:2])

In [ ]:
# Model Predictions
test_df = test_data.to_pd()
test_df["label"] = test_labels
predictions = model.predict(test)
test_df["prediction"] = predictions

test_df.sample(5)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

accuracy = accuracy_score(test_labels, predictions)
print(f'Test: {accuracy=:.4f}')

cm = confusion_matrix(test_labels, predictions)

TN = cm[0][0]
FN = cm[1][0]
TP = cm[1][1]
FP = cm[0][1]
print(f"Test: {TP=}, {TN=}, {FP=}, {FN=}")

disp = ConfusionMatrixDisplay(confusion_matrix=cm, )
disp.plot()

## Fairness
Now the fairness will be analyzed

First we import the necessary libraries

In [ ]:
import warnings
# Suppress warnings
warnings.filterwarnings("ignore")

from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
import random
from aif360.metrics import ClassificationMetric

Since for the fairness analysis, we can not use the Tabular data, we create a new dataframe with the transformed values of TabularTransform. This is because we need to have access to a dataframe structure containing the binarized values (for age, sex and all categorical attributes) and not the categorical values (e.g. Young/Aged, Female/Male etc.)

In [ ]:
column_names = transformer.get_feature_names()
# Last column is the label which is not included in the get_feature_names
column_names.append('label')

fairness_dataset = pd.DataFrame(x, columns=column_names)
fairness_dataset.head(3)

As seen below there are four columns for the protected attributes of sex and age. Two boolean columns for each:

In [ ]:
fairness_dataset[['Age_Aged', 'Age_Young', 'Sex_Male', 'Sex_Female']].head(3)

We decide only to keep `Age_Aged` and `Sex_Male`, and renaming those to columns to `Age` and `Sex` meaning that:
- when `Sex` is 0, the person is female and when when `Sex` is 1, the person is male
- when `Age` is 0, the person is young and  when `Age` is 0, the person is aged


In [ ]:
fairness_dataset = fairness_dataset.drop(columns=['Age_Young', 'Sex_Female'])
fairness_dataset.rename(columns={'Age_Aged': 'Age', 'Sex_Male': 'Sex'}, inplace=True)

Next we want to ensure that we train the classifier on the exact same data as the previous normal classifier, keeping the train, validation and test split:

In [ ]:
# Split data into training and (validation + test) datasets
train, X_temp, _, y_temp  = \
    train_test_split(fairness_dataset, fairness_dataset.label, train_size=0.70, random_state = 123)

# Split data validation and test sets
val, test, _, _ = train_test_split(X_temp, y_temp, test_size=0.5, random_state=123)

Now we can transform the previous data structures into BinaryLabelDatasets:

In [ ]:
# Transform to BinaryLabelDataset
dataset_fair_train = BinaryLabelDataset(df=train, label_names=['label'], protected_attribute_names=['Age', 'Sex'])
dataset_fair_valid = BinaryLabelDataset(df=val, label_names=['label'], protected_attribute_names=['Age', 'Sex'])
dataset_fair_test = BinaryLabelDataset(df=test, label_names=['label'], protected_attribute_names=['Age', 'Sex'])

Now the group fairness of the classifier will be assessed, assuming the protected attributes are Age, Sex.

We assume that:
- Young age (corresponding to 0) is one unprivileged group and Aged (corresponding to 1) is one privileged group
- Female sex is one unprivileged (corresponding to 0) group and male sex (corresponding to 1) is one privileged group:

In [ ]:
privileged_groups = [{'Age':1,
                      'Sex':1}]
unprivileged_groups = [{'Age': 0,
                       'Sex':0}]

Since there is a significant difference in the positive rates and true positive rate thus there is unfairness. Actually, the priviliged groups (aged and male) has 37% more chance to be predicted as >50k income comparing to the unprivileged group (young, female)

Now, we will try to mitigate this bias in the training dataset by using the `Reweighing algorithm`. By reweighing we ensure that there is better fairness when we compare the positive rates between privileged and unprivileged groups

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_fair_train)

### Fair classifier

Since reweighting does not achieve the goal of $0$ statistical parity difference or Equal opportunity difference, we will oversample the underprivileged classes (female young) and undersample the privileged classes (male aged) to teach the classifier to be more fair. After some experimentation, it was decided:
- $750$ extra samples of young female with income >50k were randomly oversampled in the training data, and random weights of values between $[1.2,1.6]$ were given for each sample
- $1500$ samples of aged male with income >50k were randomly undersampled in the training data (completely removed from the training set)

In [ ]:
# Seed to reproduce results
random.seed(1)

# Number of over-samples produced for training dataset
number_oversamples = 750
# Number of under-samples to be removed for training dataset
number_undersamples = 1500

# Train and target data
X_train = dataset_transf_train.features
y_train = dataset_transf_train.labels.ravel()

# Resampled data structures
X_train_resampled = X_train.copy()
y_train_resampled = y_train.copy()
weights_resampled = dataset_transf_train.instance_weights.copy()

# list to keep for indexes of undersamples (to be deleted after)
list_indexes_undersamples = []
# List to keep the over_samples and counters of oversamples and undersamples
oversamples = []
counter_oversamples = 0
counter_undersamples = 0

# Iterate over the whole training dataset
for counter in range(len(X_train)):
    # Only append rows that contain young female since this is the heavily underprivileged class
    if (X_train[counter][0] == 0) and (X_train[counter][1] == 0) and (y_train[counter] == 1)\
        and (counter_oversamples < number_oversamples):
        oversamples.append(X_train[counter].copy())
        counter_oversamples += 1
    # Store index of rows that contain aged male since this is the heavily overprivileged class
    if ((X_train[counter][0] == 1) or (X_train[counter][1] == 1)) and (y_train[counter] == 1)\
        and (counter_undersamples < number_undersamples):
        list_indexes_undersamples.append(counter)
        counter_undersamples += 1
    # When number of extra oversamples and undersamples is reached, stop loop
    if (counter_oversamples == number_oversamples) and (counter_undersamples == number_undersamples):
        break
    counter += 1

# Remove all indexes of undersamples
X_train_resampled = np.delete(X_train_resampled, list_indexes_undersamples, axis=0)
y_train_resampled = np.delete(y_train_resampled, list_indexes_undersamples)
weights_resampled = np.delete(weights_resampled, list_indexes_undersamples)


# Only assign positive labels to favor underprivileged class 
arr = np.array([1 for i in range(number_oversamples)])

# Append extra samples and labels to the training data and 
y_train_resampled = np.append(y_train_resampled, arr)
X_train_resampled = np.vstack([X_train_resampled, oversamples])
# Randomize weights of extra samples to take values between 1 and 1.5
weights_resampled = np.append(weights_resampled, np.array([random.uniform(1.2, 1.6) for i in range(number_oversamples)]))

Now we can train the classifier:

In [ ]:
model_fair_reweighted = RandomForestClassifier(random_state=123)
model_fair_reweighted.fit(X_train_resampled, y_train_resampled, sample_weight=weights_resampled)

X_test = dataset_fair_test.features
y_test = dataset_fair_test.labels

Next predict the test data <b> based on the threshold </b> found in the fairness analysis:

In [ ]:
predictions = []
threshold_fairness = 0.4555
# positive class index
pos_ind = 1

for item in model_fair_reweighted.predict_proba(X_test)[:,pos_ind].reshape(-1,1):
    if item >=  threshold_fairness:
        predictions.append(1)
    else:
        predictions.append(0)

Last let's print the metrics for this classifier:

In [ ]:
accuracy_fair = accuracy_score(y_test, predictions)
print(f'Test: {accuracy_fair=:.4f}')

cm = confusion_matrix(y_test, predictions)

TN = cm[0][0]
FN = cm[1][0]
TP = cm[1][1]
FP = cm[0][1]
print(f"Test: {TP=}, {TN=}, {FP=}, {FN=}")

disp = ConfusionMatrixDisplay(confusion_matrix=cm, )
disp.plot()

Also the fairness metric of `equal opportunity difference` will be printed for the fair classifier:

In [ ]:
# Preparation of predictions dateset - transform into BinaryLabelDataset with right predictions based on the fairness threshold
dataset_fair_test_pred = dataset_fair_test.copy(deepcopy=True)
dataset_fair_test_pred.scores = model_fair_reweighted.predict_proba(X_test)[:,pos_ind].reshape(-1,1)
fav_inds = dataset_fair_test_pred.scores > threshold_fairness
dataset_fair_test_pred.labels[fav_inds] = dataset_fair_test_pred.favorable_label
dataset_fair_test_pred.labels[~fav_inds] = dataset_fair_test_pred.unfavorable_label

# Calculate metric
equal_opportunity_diff_metric = ClassificationMetric(dataset_fair_test, dataset_fair_test_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups).equal_opportunity_difference()
print(f'Equal opportunity difference: {equal_opportunity_diff_metric=:.4f}')

## Privacy

To start off, let's define a new dataframe similar to the one above, the set the values of p and q for both sensitive attributes and compute for epsilon. For now, both attributes are assumed to have the same p and q, but they can be redefined to any value as seen fit.

Random response is applied to the relevant attributes afterwards.

In [ ]:
import random
import math

# randomized response 
def rand_resp(x, p=0.75, q=0.75):
    toss = random.random()
    if x == 0:
        y = 0 if toss <= q else 1
    else:
        y = 1 if toss <= p else 0
    return y

def get_epsilon(p=0.75, q=0.75):
    return math.log( max(q/(1-p), p/(1-q)) )

# apply attribute to a attribute
def privatize_attribute(column, true_label, false_label, p, q):
    # Convert labels to binary values
    binary_values = column.apply(lambda x: 1 if x == true_label else 0).values
    
    # Apply randomized response
    privatized_values = pd.Series([rand_resp(x, p, q) for x in binary_values], index=column.index)

    # Convert back to original labels
    return privatized_values.apply(lambda x: true_label if x == 1 else false_label)

# Create a copy of the original data
df_private = df.copy(deep=True)

# Set values of p and q
p_age, q_age = 0.8, 0.8
p_sex, q_sex = 0.8, 0.8  

epsilon_age = get_epsilon(p_age, q_age)
epsilon_sex = get_epsilon(p_sex, q_sex)
print(f"We will apply {epsilon_age:.3f}-LDP setting p={p_age}, q={q_age} for age \
AND {epsilon_sex:.3f}-LDP setting p={p_sex}, q={q_sex} for sex.")

# Apply randomized response to Age and Sex
df_private['Age'] = privatize_attribute(df_private['Age'], 'Aged', 'Young', p_age, q_age)
df_private['Sex'] = privatize_attribute(df_private['Sex'], 'Male', 'Female', p_sex, q_sex)

# Display the new DataFrame
df_private.head()

We compute the cross-tabulation on the original data and private data to estimate how many people exist in value combinations of the two sensitive attributes.

In [ ]:
from tabulate import tabulate

# Showing cross tabulation
print("\nOriginal Cross-tabulation:")
original_crosstab = pd.crosstab(df['Age'], df['Sex'])
print(tabulate(original_crosstab, headers='keys', tablefmt='pretty'))

print("\nPrivatized Cross-tabulation:")
privatized_crosstab = pd.crosstab(df_private['Age'], df_private['Sex'])
print(tabulate(privatized_crosstab, headers='keys', tablefmt='pretty'))

Compute the error between the privatized and the original data.

In [ ]:
def pct_error(true_value, estimated_value):

    if true_value == 0:
        # If the true value is zero, the percent error is undefined.
        return float('inf')
    
    # Calculate the absolute difference
    abs_error = abs(true_value - estimated_value)
        
    # Calculate the percent error
    pct_error = (abs_error / true_value) * 100
        
    return pct_error


error_female_young = pct_error(original_crosstab["Female"]["Young"], privatized_crosstab["Female"]["Young"])
print(f"The percent error for young female responses is: {error_female_young}%")

error_female_aged = pct_error(original_crosstab["Female"]["Aged"], privatized_crosstab["Female"]["Aged"])
print(f"The percent error for aged female responses is: {error_female_aged}%")

error_male_young = pct_error(original_crosstab["Male"]["Young"], privatized_crosstab["Male"]["Young"])
print(f"The percent error for young male responses is: {error_male_young}%")

error_male_aged = pct_error(original_crosstab["Male"]["Aged"], privatized_crosstab["Male"]["Aged"])
print(f"The percent error for aged male responses is: {error_male_aged}%")

Estimating the actual responses.

In [ ]:
def estimate(column, p=0.75, q=0.75):
    n_people = len(column)
    n_reported = np.sum(column.astype(int))
    return (n_reported/n_people + q - 1)/(p+q-1)*n_people

# insert estimate here

In [ ]:
# Implement Private Classifier

tabular_data_private = Tabular(
   df_private,
   categorical_columns=[
  "Age", "Workclass", "Education", "Occupation", "Relationship", "Race", "Sex", "Hours per week", "Country"
],
   target_column='label'
)
transformer_private = TabularTransform().fit(tabular_data_private)
class_names = transformer_private.class_names
x_private = transformer_private.transform(tabular_data_private)

# Split data into training and (validation + test) datasets
train_private, X_private_temp, train_labels_private, y_private_temp  = \
    train_test_split(x_private[:, :-1], x_private[:, -1], train_size=0.70, random_state = 123)

# Split data validation and test sets
val_private, test_private, val_labels_private, test_labels_private = train_test_split(X_private_temp, y_private_temp, test_size=0.5, random_state=123)

test_labels_private = test_labels_private.astype(int)

print('Private Classfier')
print('Training data shape:   {}'.format(train_private.shape))
print('Validation data shape:  {}'.format(val_private.shape))
print('Test data shape:        {}'.format(test_private.shape))

# Train a Random Forest model
model_private = RandomForestClassifier(random_state=123)
model_private.fit(train_private, train_labels_private)

predict_function_private=lambda z: model_private.predict_proba(transformer_private.transform(z))

# # Convert the transformed data back to Tabular instances
train_data_private = transformer_private.invert(train_private)
test_data_private = transformer_private.invert(test_private)

display(tabular_data_private.target_column)
display(train_labels_private[:2])

In [ ]:
# Model Predictions
test_df_private = test_data_private.to_pd()
test_df_private["label"] = test_labels_private
predictions_private = model_private.predict(test_private)
test_df_private["prediction"] = predictions_private

test_df_private.sample(5)

In [ ]:
accuracy_private = accuracy_score(test_labels_private, predictions_private)
print(f'Test: {accuracy_private=:.4f}')

cm_private = confusion_matrix(test_labels_private, predictions_private)

TN = cm_private[0][0]
FN = cm_private[1][0]
TP = cm_private[1][1]
FP = cm_private[0][1]
print(f"Test: {TP=}, {TN=}, {FP=}, {FN=}")

disp = ConfusionMatrixDisplay(confusion_matrix=cm_private, )
disp.plot()

The accuracy lowered as expected. Implementing differential privacy involves adding noise to the training process to protect individual data points. This added noise can degrade the model's accuracy, especially as epsilon decreases. Lower epsilon values correspond to stronger privacy guarantees but also result in greater noise, which can lead to decreased model performance.

## Privacy and Fairness
Now a fair version of the private classifier will be analyzed

We use the private data set from earlier and transform it to be compatible with the fairness classifier

In [ ]:
np.random.seed(123)
random.seed(123)

# Step 1: Assign feature names and create DataFrame
column_names_pf = transformer_private.get_feature_names()
column_names_pf.append('label')  # Add the label column
privatized_data_fair = pd.DataFrame(x, columns=column_names_pf)
privatized_data_fair.head(3)

In [ ]:
privatized_data_fair[['Age_Aged', 'Age_Young', 'Sex_Male', 'Sex_Female']].head(3)

In [ ]:
privatized_data_fair = privatized_data_fair.drop(columns=['Age_Young', 'Sex_Female'])
privatized_data_fair.rename(columns={'Age_Aged': 'Age', 'Sex_Male': 'Sex'}, inplace=True)

In [ ]:
# Split into train, validation, and test sets
train_priv, X_temp_priv, _, y_temp_priv = train_test_split(
    privatized_data_fair, privatized_data_fair.label, train_size=0.70, random_state=123
)
val_priv, test_priv, _, _ = train_test_split(X_temp_priv, y_temp_priv, test_size=0.5, random_state=123)

In [ ]:
# Create BinaryLabelDataset for privatized data
dataset_priv_train = BinaryLabelDataset(
    df=train_priv, label_names=['label'], protected_attribute_names=['Age', 'Sex']
)
dataset_priv_valid = BinaryLabelDataset(
    df=val_priv, label_names=['label'], protected_attribute_names=['Age', 'Sex']
)
dataset_priv_test = BinaryLabelDataset(
    df=test_priv, label_names=['label'], protected_attribute_names=['Age', 'Sex']
)

### Reweighing
Use the Reweighing technique on the privatized dataset to adjust for fairness.

In [ ]:
privileged_groups = [{'Age':1,
                      'Sex':1}]
unprivileged_groups = [{'Age': 0,
                       'Sex':0}]
# Apply Reweighing
RW_priv = Reweighing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)
dataset_transf_priv_train = RW_priv.fit_transform(dataset_priv_train)

### Resampling
Resample the data to correct for underprivileged and overprivileged classes.

In [ ]:
# Train and target data
X_train_priv = dataset_transf_priv_train.features
y_train_priv = dataset_transf_priv_train.labels.ravel()
weights_priv = dataset_transf_priv_train.instance_weights.copy()

# Resampling parameters
number_oversamples = 750
number_undersamples = 1500

# Resampling logic (use the same logic from Part 2)
oversamples_priv = []
undersamples_indexes_priv = []
counter_oversamples_priv = 0
counter_undersamples_priv = 0

for i in range(len(X_train_priv)):
    # Over-sample underprivileged group (Young Female, 0,0)
    if (X_train_priv[i][0] == 0) and (X_train_priv[i][1] == 0) and (y_train_priv[i] == 1) \
            and counter_oversamples_priv < number_oversamples:
        oversamples_priv.append(X_train_priv[i].copy())
        counter_oversamples_priv += 1
    # Under-sample overprivileged group (Aged Male, 1,1)
    if ((X_train_priv[i][0] == 1) or (X_train_priv[i][1] == 1)) and (y_train_priv[i] == 1) \
            and counter_undersamples_priv < number_undersamples:
        undersamples_indexes_priv.append(i)
        counter_undersamples_priv += 1
    # Stop when limits are reached
    if counter_oversamples_priv == number_oversamples and counter_undersamples_priv == number_undersamples:
        break

# Remove under-samples
X_train_priv_resampled = np.delete(X_train_priv, undersamples_indexes_priv, axis=0)
y_train_priv_resampled = np.delete(y_train_priv, undersamples_indexes_priv)
weights_priv_resampled = np.delete(weights_priv, undersamples_indexes_priv)

# Add over-samples
X_train_priv_resampled = np.vstack([X_train_priv_resampled, oversamples_priv])
y_train_priv_resampled = np.append(y_train_priv_resampled, [1] * len(oversamples_priv))
weights_priv_resampled = np.append(weights_priv_resampled, np.random.uniform(1.2, 1.6, len(oversamples_priv)))


### Train Privacy + Fairness Classifier
Train a classifier using the resampled, reweighted, and privacy-preserved dataset.


In [ ]:
# Train a Random Forest model
model_priv_fair = RandomForestClassifier(random_state=123)
model_priv_fair.fit(X_train_priv_resampled, y_train_priv_resampled, sample_weight=weights_priv_resampled)

# Test data
X_test_priv = dataset_priv_test.features
y_test_priv = dataset_priv_test.labels

# Predictions
predictions_priv_fair = []
threshold_priv_fair = 0.4555  # Example threshold
pos_ind = 1  # Positive class index

for prob in model_priv_fair.predict_proba(X_test_priv)[:, pos_ind]:
    predictions_priv_fair.append(1 if prob >= threshold_priv_fair else 0)

### Evaluate Privacy + Fairness
Accuracy
Evaluate the accuracy of the classifier.

In [ ]:
accuracy_priv_fair = accuracy_score(y_test_priv, predictions_priv_fair)
print(f'Test Accuracy (Privacy + Fairness): {accuracy_priv_fair:.4f}')

Confusion Matrix

In [ ]:
cm_priv_fair = confusion_matrix(y_test_priv, predictions_priv_fair)
disp_priv_fair = ConfusionMatrixDisplay(confusion_matrix=cm_priv_fair)
disp_priv_fair.plot()

Fairness Metric calculated using the real age and sex values


In [ ]:
dataset_priv_test_pred = dataset_priv_test.copy(deepcopy=True)
dataset_priv_test_pred.scores = model_priv_fair.predict_proba(X_test_priv)[:, pos_ind].reshape(-1, 1)

fav_inds_priv = dataset_priv_test_pred.scores > threshold_priv_fair
dataset_priv_test_pred.labels[fav_inds_priv] = dataset_priv_test_pred.favorable_label
dataset_priv_test_pred.labels[~fav_inds_priv] = dataset_priv_test_pred.unfavorable_label

eod_priv_fair = ClassificationMetric(
    # dataset_priv_test
    dataset_fair_test
    , dataset_priv_test_pred,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
).equal_opportunity_difference()

print(f'Equal Opportunity Difference (Privacy + Fairness): {eod_priv_fair:.4f}')

In [ ]:
# Example values (replace with actual results)
models = ['Fairness-Only', 'Privacy+Fairness']
# accuracy_met=accuracy
accuracy = [accuracy_fair, accuracy_priv_fair]
fairness = [equal_opportunity_diff_metric, eod_priv_fair]  # EOD values (lower is better)

# Scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(fairness, accuracy, color=['green', 'orange'], s=100, label=models)

# Annotate points
for i, model in enumerate(models):
    plt.text(fairness[i] + 0.01, accuracy[i], model, fontsize=10)

# Labels and title
plt.xlabel('Fairness (EOD)')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Fairness Trade-off')
plt.axhline(y=max(accuracy), color='gray', linestyle='--', label='Max Accuracy')
plt.axvline(x=min(fairness), color='red', linestyle='--', label='Max Fairness')
plt.legend()
plt.grid(True)
plt.show()

As expected the accuracy of the fairness only classifier is higher because the model is trained on the unaltered data. On the other hand, adding privacy via the Local Differential Privacy introduces noise to sensitive attributes, which leads to less accurate models because the underlying data quality is degraded. When combined with fairness adjustments, the performance further decreases because these adjustments reduce bias but may also reduce model fit. Moreover, the noise from the privacy mechanisms adds noise to sensitive attributes. 
This noise:

1- Makes it harder to accurately distinguish between privileged and unprivileged groups.

2- Could lead to unintentional oversampling or misclassification of the unprivileged group as privileged during fairness adjustments, or vice versa.

3- If the noise disproportionately affects the privileged group, it may unintentionally boost the TPR for the unprivileged group.